Since [OpenPaths](https://openpaths.cc/) is dead, we switched to [WHIB](http://www.bleatinc.com/), which wasn't [owned by Facebook](https://moves-app.com/) (or actually also dead now) and doesn't [drain the battery of my phone quickly](http://www.bigpaua.com/arc/).
A 2$ *premium* feature lets you export all the data as CSV, which is easy to parse and display.

In [1]:
import glob
import pandas
import geopy
import geopy.distance
import geopy.geocoders
import folium
import folium.plugins

In [2]:
# Which year do we want to look at?
whichyear = 2019

In [3]:
# Map details
# Available tiles: https://github.com/python-visualization/folium/blob/master/folium/folium.py#L75
#tileprovider = 'Mapbox Bright'
tileprovider = 'Cartodb positron'
#tileprovider = 'Cartodb dark_matter'
#tileprovider = 'Mapbox Control Room'
# tileprovider = 'Stamen Toner'
# Default zoom level and circle marker radius
zoom_start = 5
radius = 10

In [4]:
# Settings for address lookup
#geopy.geocoders.options.default_user_agent = 'Jahresrückblick Habi'
geolocator = geopy.geocoders.Nominatim()

/Users/habi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
# Read in locations from newest CSV-file in the current directory
locations = pandas.read_csv(sorted(glob.glob('journey*.csv'))[-1])

In [6]:
# Modify the dataframe
locations.drop(['Crumb'], axis=1, inplace=True)
locations.rename(columns={'LocalDate': 'Date'}, inplace=True)
locations.rename(columns={'LocalTime': 'Time'}, inplace=True)
locations.rename(columns={'Altitude (in metres)': 'Altitude'}, inplace=True)
locations.rename(columns={'Accuracy (in metres)': 'Accuray'}, inplace=True)

In [7]:
# Make us a proper date column, based on https://stackoverflow.com/a/26763793
locations['Date'] = pandas.to_datetime(locations['Date'])
# Make us a year, month and weekday colum, based on https://stackoverflow.com/q/48623332
locations['Year'] = locations.Date.dt.year
locations['Month'] = locations.Date.dt.month
locations['Day'] = locations.Date.dt.day
locations['Weekday'] = locations.Date.dt.dayofweek

In [8]:
# Drop all values not in 'whichyear'
# https://stackoverflow.com/a/27360130
# locations.drop(locations[locations.Year != whichyear].index, inplace=True)

In [9]:
# Drop all values not in a certain month
# locations.drop(locations[locations.Month != 7].index, inplace=True)

In [10]:
locations.head()

,Date,Time,Latitude,Longitude,Altitude,Accuray,Year,Month,Day,Weekday
0,2018-03-02,23:37,46.935401,7.418162,553,65,2018,3,2,4
1,2018-04-02,09:01,46.935344,7.417897,553,50,2018,4,2,0
2,2018-04-02,09:02,46.932949,7.422317,553,57,2018,4,2,0
3,2018-04-02,09:12,46.932855,7.422384,565,38,2018,4,2,0
4,2018-04-02,09:14,46.923639,7.415233,565,61,2018,4,2,0


In [11]:
locations.tail()

,Date,Time,Latitude,Longitude,Altitude,Accuray,Year,Month,Day,Weekday
24960,2019-05-08,13:42,46.935573,7.418067,0,0,2019,5,8,2
24961,2019-05-08,13:42,46.935573,7.418067,0,0,2019,5,8,2
24962,2019-05-08,13:47,46.935378,7.418065,551,20,2019,5,8,2
24963,2019-05-08,13:49,46.935408,7.418112,550,65,2019,5,8,2
24964,2019-05-08,13:50,46.935424,7.418078,551,102,2019,5,8,2


In [12]:
locations.describe()

,Latitude,Longitude,Altitude,Accuray,Year,Month,Day,Weekday
count,24965.000000,24965.000000,24965.000000,24965.000000,24965.000000,24965.000000,24965.000000,24965.000000
mean,46.709087,7.627716,557.487242,199.463008,2018.445504,6.201682,15.879952,3.086241
std,1.333904,0.602871,448.511634,1782.746561,0.497031,3.332899,9.004475,1.950880
min,39.118591,4.330553,-143.000000,0.000000,2018.000000,1.000000,1.000000,0.000000
25%,46.930673,7.418067,499.000000,38.000000,2018.000000,3.000000,8.000000,1.000000
50%,46.938454,7.428865,547.000000,65.000000,2018.000000,6.000000,16.000000,3.000000
75%,46.948900,7.456851,557.000000,83.000000,2019.000000,9.000000,24.000000,5.000000
max,51.033511,10.433116,3062.000000,149000.000000,2019.000000,12.000000,31.000000,6.000000


In [13]:
# Show the extreme locations on a map
# Marker colors from here: https://stackoverflow.com/a/41993318
m = folium.Map(location=[locations['Latitude'].mean(),
                         locations['Longitude'].mean()],
               tiles=tileprovider,
               zoom_start=zoom_start)
# Altitude
for c, loc in locations.sort_values(by=['Altitude'], ascending=False).head(1).iterrows():
    folium.CircleMarker(location=[loc.Latitude, loc.Longitude],
                        radius=radius,
                        popup='Highest: %s' % (loc.Date),
                       ).add_to(m)
# North
for c, loc in locations.sort_values(by=['Latitude'], ascending=False).head(1).iterrows():
    folium.CircleMarker(location=[loc.Latitude, loc.Longitude],
                        radius=radius,
                        popup='North: %s' % (loc.Date),
                       ).add_to(m)
# East
for c, loc in locations.sort_values(by=['Longitude'], ascending=False).head(1).iterrows():
    folium.CircleMarker(location=[loc.Latitude, loc.Longitude],
                        radius=radius,
                        popup='East: %s' % (loc.Date),
                       ).add_to(m)
# South
for c, loc in locations.sort_values(by=['Latitude']).head(1).iterrows():
    folium.CircleMarker(location=[loc.Latitude, loc.Longitude],
                        radius=radius,
                        popup='South: %s' % (loc.Date),
                       ).add_to(m)
# West
for c, loc in locations.sort_values(by=['Longitude']).head(1).iterrows():
    folium.CircleMarker(location=[loc.Latitude, loc.Longitude],
                        radius=radius,
                        popup='West: %s' % (loc.Date),
                       ).add_to(m)
m

In [14]:
# Show north, east, south, west extreme (with averaged lat/lon for each)
m = folium.Map(location=[locations['Latitude'].mean(),
                         locations['Longitude'].mean()],
               tiles=tileprovider,
               zoom_start=zoom_start)
folium.CircleMarker(location=[locations.Latitude.mean(), locations.Longitude.min()],
                    radius=radius,
                    popup='Average West').add_to(m)
folium.CircleMarker(location=[locations.Latitude.mean(), locations.Longitude.max()],
                    radius=radius,
                    popup='Average East').add_to(m)
folium.CircleMarker(location=[locations.Latitude.max(), locations.Longitude.mean()],
                    radius=radius,
                    popup='Average North').add_to(m)
folium.CircleMarker(location=[locations.Latitude.min(), locations.Longitude.mean()],
                    radius=radius,
                    popup='Average South').add_to(m)
m

In [15]:
# How far did we come?
print('In %s, we traveled %0.0f km north-south' % (whichyear,
                                                   geopy.distance.geodesic([locations.Latitude.min(), locations.Longitude.mean()],
                                                                         [locations.Latitude.max(), locations.Longitude.mean()]).km))
print('In %s, we traveled %0.0f km east-west' % (whichyear,
                                                 geopy.distance.geodesic([locations.Latitude.mean(), locations.Longitude.min()],
                                                                         [locations.Latitude.mean(), locations.Longitude.max()]).km))


In 2019, we traveled 1324 km north-south
In 2019, we traveled 467 km east-west


By using [the geocoding library for Python](https://github.com/geopy/geopy) and the [OpenStreetMap reverse geocoding tool](https://wiki.openstreetmap.org/wiki/Nominatim) we can assign addresses to locations.

In [16]:
# Get all extrema locations
location_average = [locations['Latitude'].mean(), locations['Longitude'].mean()]
location_median = [locations['Latitude'].median(), locations['Longitude'].median()]
location_north = [locations[locations.Latitude == locations.Latitude.max()].Latitude.values[0],
                  locations[locations.Latitude == locations.Latitude.max()].Longitude.values[0]]
location_east = [locations[locations.Longitude == locations.Longitude.max()].Latitude.values[0],
                 locations[locations.Longitude == locations.Longitude.max()].Longitude.values[0]]
location_south = [locations[locations.Latitude == locations.Latitude.min()].Latitude.values[0],
                  locations[locations.Latitude == locations.Latitude.min()].Longitude.values[0]]
location_west = [locations[locations.Longitude == locations.Longitude.min()].Latitude.values[0],
                 locations[locations.Longitude == locations.Longitude.min()].Longitude.values[0]]
location_top = [locations[locations['Altitude'] == locations['Altitude'].max()].Latitude.values[0],
                locations[locations['Altitude'] == locations['Altitude'].max()].Longitude.values[0]]

In [17]:
# Turn debug on to print and thus find the right value of the address
debug = False

In [18]:
# Address lookup with `geopy` and https://wiki.openstreetmap.org/wiki/Nominatim
location_average_geo = geolocator.reverse(location_average)
# Print the details, so we can get the correct value to save below
if debug:
    print(location_average_geo.raw)
# Save us a name and print it
name_average = location_average_geo.raw.get('address').get('village')
print('The average location in %s was in %s' % (whichyear, name_average))
# Show the point on a map
m = folium.Map(location=[float(location_average_geo.raw.get('lat')),
                         float(location_average_geo.raw.get('lon'))],
               tiles=tileprovider,
               zoom_start=zoom_start*3)
folium.CircleMarker(location=[float(location_average_geo.raw.get('lat')),
                              float(location_average_geo.raw.get('lon'))],
                    radius=radius,
                    popup=name_average).add_to(m)
m

The average location in 2019 was in Zwieselberg


In [19]:
location_median_geo = geolocator.reverse(location_median)
if debug:
    print(location_median_geo.raw)
name_median = location_median_geo.raw.get('address').get('suburb')
print('The median location in %s was in %s' % (whichyear, name_median))
m = folium.Map(location=[float(location_median_geo.raw.get('lat')),
                         float(location_median_geo.raw.get('lon'))],
               tiles=tileprovider,
               zoom_start=zoom_start*3)
folium.CircleMarker(location=[float(location_median_geo.raw.get('lat')),
                              float(location_median_geo.raw.get('lon'))],
                    radius=radius,
                    popup=name_median).add_to(m)
m

The median location in 2019 was in Weissenbühl


In [20]:
location_north_geo = geolocator.reverse(location_north)
if debug:
    print(location_north_geo.raw)
name_north = location_north_geo.raw.get('address').get('city')
print('The northmost location in %s was in %s' % (whichyear, name_north))
m = folium.Map(location=[float(location_north_geo.raw.get('lat')),
                         float(location_north_geo.raw.get('lon'))],
               tiles=tileprovider,
               zoom_start=zoom_start*3)
folium.CircleMarker(location=[float(location_north_geo.raw.get('lat')),
                              float(location_north_geo.raw.get('lon'))],
                    radius=radius,
                    popup=name_north).add_to(m)
m

The northmost location in 2019 was in None


In [21]:
location_east_geo = geolocator.reverse(location_east)
if debug:
    print(location_east_geo.raw)
name_east = location_east_geo.raw.get('address').get('town')
print('The most eastern location in %s was in %s' % (whichyear, name_east))
m = folium.Map(location=[float(location_east_geo.raw.get('lat')),
                         float(location_east_geo.raw.get('lon'))],
               tiles=tileprovider,
               zoom_start=zoom_start*3)
folium.CircleMarker(location=[float(location_east_geo.raw.get('lat')),
                              float(location_east_geo.raw.get('lon'))],
                    radius=radius,
                    popup=name_east).add_to(m)
m

The most eastern location in 2019 was in None


In [22]:
location_south_geo = geolocator.reverse(location_south)
if debug:
    print(location_south_geo.raw)
name_south = location_south_geo.raw.get('address').get('beach')
print('The southmost location in %s was at the %s' % (whichyear, name_south))
m = folium.Map(location=[float(location_south_geo.raw.get('lat')),
                         float(location_south_geo.raw.get('lon'))],
               tiles=tileprovider,
               zoom_start=zoom_start*3)
folium.CircleMarker(location=[float(location_south_geo.raw.get('lat')),
                              float(location_south_geo.raw.get('lon'))],
                    radius=radius,
                    popup=name_south).add_to(m)
m

The southmost location in 2019 was at the Spiaggia di Porto Giunco


In [23]:
location_west_geo = geolocator.reverse(location_west)
if debug:
    print(location_west_geo.raw)
name_west = location_west_geo.raw.get('address').get('town')
print('The most western location in %s was in %s' % (whichyear, name_west))
m = folium.Map(location=[float(location_west_geo.raw.get('lat')),
                         float(location_west_geo.raw.get('lon'))],
               tiles=tileprovider,
               zoom_start=zoom_start*3)
folium.CircleMarker(location=[float(location_west_geo.raw.get('lat')),
                              float(location_west_geo.raw.get('lon'))],
                    radius=radius,
                    popup=name_west).add_to(m)
m

The most western location in 2019 was in Anderlecht


In [24]:
location_top_geo = geolocator.reverse(location_top)
if debug:
    print(location_top_geo.raw)
name_top = location_top_geo.raw.get('address').get('path')
print('The highest location in %s was on the %s with %s masl.' % (whichyear,
                                                                  name_top,
                                                                  locations[locations['Altitude'] == locations['Altitude'].max()].Altitude.values[0]))
m = folium.Map(location=[float(location_top_geo.raw.get('lat')),
                         float(location_top_geo.raw.get('lon'))],
               tiles=tileprovider,
               zoom_start=zoom_start*3)
folium.CircleMarker(location=[float(location_top_geo.raw.get('lat')),
                              float(location_top_geo.raw.get('lon'))],
                    radius=radius,
                    popup=name_top).add_to(m)
m

The highest location in 2019 was on the None with 3062 masl.


In [25]:
# Plot *all* locations on a single map
m = folium.Map(location=[locations['Latitude'].mean(),
                         locations['Longitude'].mean()],
               tiles=tileprovider,
               zoom_start=zoom_start)
showextremes = False
if showextremes:
    # Extreme locations
    foliuum.CircleMarker(location=location_average,
                        radius=radius,
                        popup='Average location in %s' % name_average).add_to(m)
    folium.CircleMarker(location=location_north,
                        radius=radius,
                        popup='Northmost location in %s' % name_north).add_to(m)
    folium.CircleMarker(location=location_east,
                        radius=radius,
                        popup='Northmost location in %s' % name_east).add_to(m)
    folium.CircleMarker(location=location_south,
                        radius=radius,
                        popup='Northmost location in %s' % name_south).add_to(m)
    folium.CircleMarker(location=location_west,
                        radius=radius,
                        popup='Northmost location in %s' % name_west).add_to(m)
    folium.CircleMarker(location=location_top,
                        radius=radius,
                        popup='Highest location (%s m) on the %s' % (locations[locations['Altitude'] == locations['Altitude'].max()].Altitude.values[0],
                                                                     name_top)).add_to(m)
# All locations, in different ways
singlepoints = False
fast = True
if singlepoints:
    for c, loc in locations.iterrows():
        # not every point, but every x-th one, or else the map is too slow
        if not c % 1:
            folium.CircleMarker(location=[loc.Latitude, loc.Longitude],
                                radius=2,
                                popup='%s@%s' % (loc.Date, loc.Time),
                                color='darkred'
                               ).add_to(m)
else:
    if fast:
        # FastMarkerCluster
        m.add_child(folium.plugins.FastMarkerCluster(locations[['Latitude', 'Longitude']].values.tolist()))
    else:
        # Markercluster
        mc = folium.plugins.MarkerCluster()
        for c, loc in locations.iterrows():
            mc.add_child(folium.CircleMarker(location=[loc.Latitude, loc.Longitude],
                                             popup='%s@%s' % (loc.Date, loc.Time)))
        m.add_child(mc)
m.save('map-points.html')
m

In [26]:
# Viridis colormap from here: https://www.thedataschool.co.uk/gwilym-lockwood/viridis-colours-tableau/
gradient={0.00: '#440154FF',
          0.05: '#481567FF',
          0.10: '#482677FF',
          0.15: '#453781FF',
          0.20: '#404788FF',
          0.25: '#39568CFF',
          0.30: '#33638DFF',
          0.35: '#2D708EFF',
          0.40: '#287D8EFF',
          0.45: '#238A8DFF',
          0.50: '#1F968BFF',
          0.55: '#20A387FF',
          0.60: '#29AF7FFF',
          0.65: '#3CBB75FF',
          0.70: '#55C667FF',
          0.75: '#73D055FF',
          0.80: '#95D840FF',
          0.85: '#B8DE29FF',
          0.90: '#DCE319FF',
          0.95: '#FDE725FF'}

In [27]:
# Show a heatmap instead of single points
m = folium.Map(location=[locations['Latitude'].mean(),
                         locations['Longitude'].mean()],
               tiles=tileprovider,
               zoom_start=zoom_start)
if showextremes:
    # Extreme locations
    folium.CircleMarker(location=location_average,
                        radius=radius,
                        popup='Average location in %s' % name_average,
                       ).add_to(m)
    folium.CircleMarker(location=location_north,
                        radius=radius,
                        popup='Northmost location in %s' % name_north,
                       ).add_to(m)
    folium.CircleMarker(location=location_east,
                        radius=radius,
                        popup='Most eastern location in %s' % name_east,
                       ).add_to(m)
    folium.CircleMarker(location=location_south,
                        radius=radius,
                        popup='Southmost location in %s' % name_south,
                       ).add_to(m)
    folium.CircleMarker(location=location_west,
                        radius=radius,
                        popup='Most western location in %s' % name_west,
                       ).add_to(m)
    folium.CircleMarker(location=location_top,
                        radius=radius,
                        popup='Highest location (%s m) on the %s' % (locations[locations['Altitude'] == locations['Altitude'].max()].Altitude.values[0],
                                                                     name_top),
                       ).add_to(m)
# Add heatmap
folium.plugins.HeatMap(locations[['Latitude', 'Longitude']].values.tolist(),
                       gradient=gradient, blur=0).add_to(m)
m.save('map-heat.html')
m